In [1]:
import os
import pandas as pd

datapath = "../data"

In [2]:
from rdkit import Chem
from standardiser import standardise
import numpy as np

def standardise_smiles(smiles):
    st_smiles = []
    for smi in smiles:
        if smi is None:
            st_smi = np.nan
            st_smiles += [st_smi]
            continue
        smi = str(smi)
        smi = smi.strip()
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            st_smi = np.nan
            st_smiles += [st_smi]
            continue
        try:
            std_mol = standardise.run(mol)
            st_smi = Chem.MolToSmiles(std_mol, canonical=True)
            st_smiles += [st_smi]
        except:
            st_smi = Chem.MolToSmiles(mol, canonical=True)
            st_smiles += [st_smi]
        if std_mol is None:
            st_smi = Chem.MolToSmiles(mol, canonical=True)
            st_smiles += [st_smi]
            continue
    return st_smiles

# Chembl325 data

The data was downloaded on 19th Nov 2025 from ChEMBL, by selecting all IC50 available data on the target chembl325

In [17]:
df = pd.read_csv(os.path.join(datapath, "chembl325_ic50.csv"),low_memory=False)
df.columns

Index(['Molecule ChEMBL ID', 'Molecule Name', 'Molecule Max Phase',
       'Molecular Weight', '#RO5 Violations', 'AlogP', 'Compound Key',
       'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value',
       'Standard Units', 'pChEMBL Value', 'Data Validity Comment', 'Comment',
       'Uo Units', 'Ligand Efficiency BEI', 'Ligand Efficiency LE',
       'Ligand Efficiency LLE', 'Ligand Efficiency SEI', 'Potential Duplicate',
       'Assay ChEMBL ID', 'Assay Description', 'Assay Type', 'BAO Format ID',
       'BAO Label', 'Assay Organism', 'Assay Tissue ChEMBL ID',
       'Assay Tissue Name', 'Assay Cell Type', 'Assay Subcellular Fraction',
       'Assay Parameters', 'Assay Variant Accession', 'Assay Variant Mutation',
       'Target ChEMBL ID', 'Target Name', 'Target Organism', 'Target Type',
       'Document ChEMBL ID', 'Source ID', 'Source Description',
       'Document Journal', 'Document Year', 'Cell ChEMBL ID', 'Properties',
       'Action Type', 'Standard Text Value', 'V

In [18]:
print(len(df[df["pChEMBL Value"].isna()]))
print(len(df[df["Smiles"].isna()]))
df["st_smiles"] = standardise_smiles(df["Smiles"].tolist())
print(len(df[df["st_smiles"].isna()]))
df = df[~df["pChEMBL Value"].isna()]
df = df[~df["st_smiles"].isna()]
df = df[["st_smiles", "pChEMBL Value"]]
print(len(df))

1814
8


[12:44:43] SMILES Parse Error: syntax error while parsing: nan
[12:44:43] SMILES Parse Error: check for mistakes around position 2:
[12:44:43] nan
[12:44:43] ~^
[12:44:43] SMILES Parse Error: Failed parsing SMILES 'nan' for input: 'nan'
[12:44:46] SMILES Parse Error: syntax error while parsing: nan
[12:44:46] SMILES Parse Error: check for mistakes around position 2:
[12:44:46] nan
[12:44:46] ~^
[12:44:46] SMILES Parse Error: Failed parsing SMILES 'nan' for input: 'nan'
[12:44:49] SMILES Parse Error: syntax error while parsing: nan
[12:44:49] SMILES Parse Error: check for mistakes around position 2:
[12:44:49] nan
[12:44:49] ~^
[12:44:49] SMILES Parse Error: Failed parsing SMILES 'nan' for input: 'nan'
[12:44:53] SMILES Parse Error: syntax error while parsing: nan
[12:44:53] SMILES Parse Error: check for mistakes around position 2:
[12:44:53] nan
[12:44:53] ~^
[12:44:53] SMILES Parse Error: Failed parsing SMILES 'nan' for input: 'nan'
[12:44:53] SMILES Parse Error: syntax error while pa

8
9892


In [20]:
df_ = df.copy()
df_["bin"] = [1 if x >= 7 else 0 for x in df_["pChEMBL Value"].tolist()]
print("Cut-off pIC50 7: total: ", len(df_), " actives: ", len(df_[df_["bin"]==1]))
df_ = df_[["st_smiles", "bin"]]
df_.to_csv(os.path.join(datapath, "ml_datasets", "chembl325_pIC50_7.csv"), index=False)

Cut-off pIC50 7: total:  9892  actives:  3817


In [19]:
df_ = df.copy()
df_["bin"] = [1 if x >= 8 else 0 for x in df_["pChEMBL Value"].tolist()]
print("Cut-off pIC50 8: total: ", len(df_), " actives: ", len(df_[df_["bin"]==1]))
df_ = df_[["st_smiles", "bin"]]
df_.to_csv(os.path.join(datapath, "ml_datasets", "chembl325_pIC50_8.csv"), index=False)

Cut-off pIC50 8: total:  9892  actives:  1210
